# Closures in Julia

Many languages support closures. Today I needed to run part of my program for up to an hour.
Instead of putting the timer in the same block of code, I wanted to separate the responsibility. Closures seemed like a good idea.

## Dummy closure


In [ ]:
function withDummy(func::Function)
    println("[DEBUG] Before")
    func()
    println("[DEBUG] After")
end

In [2]:
withDummy() do
    println("Hi")
end

[DEBUG] Before
Hi
[DEBUG] After


## The timer closure

Define a function that will take responsibility for stopping the code after the specified time.
One thing to note, if the function body finishes before the specified time, we will still have to wait for it.

In [1]:
function runFor(body::Function, seconds::Int64)
    println("[DEBUG] Beginning async task")
    task = @async (
        try
            body()
        catch e
            if isa(e, InterruptException)
                println("[DEBUG] InteruptException caught, shutting down")
            else
                rethrow(e)
            end
        end
    )
    println("[DEBUG] Waiting for timer to expire")
    sleep(seconds)
    println("[DEBUG] Stopping the async task with InteruptException")
    @async Base.throwto(task, InterruptException()) # I'm not sure why, but it doesn't seem to kill the task without @async
    println("[DEBUG] Exiting")
    return
end

runFor (generic function with 1 method)

Now run a block of code inside the closure:

In [2]:
for i = 1:3
    println("before the long task")
    
    runFor(5 #= seconds =#) do
        while true
            println("running computationally expensive task")
            sleep(1)
        end
    end
    
    println("after the long task")
end

before the long task
[DEBUG] Beginning async task
[DEBUG] Waiting for timer to expire
running computationally expensive task
running computationally expensive task
running computationally expensive task
running computationally expensive task
running computationally expensive task
[DEBUG] Stopping the async task with InteruptException
[DEBUG] Exiting
after the long task
before the long task
[DEBUG] Beginning async task
[DEBUG] Waiting for timer to expire
[DEBUG] InteruptException caught, shutting down
running computationally expensive task
running computationally expensive task
running computationally expensive task
running computationally expensive task
running computationally expensive task
[DEBUG] Stopping the async task with InteruptException
[DEBUG] Exiting
after the long task
before the long task
[DEBUG] Beginning async task
[DEBUG] Waiting for timer to expire
[DEBUG] InteruptException caught, shutting down
running computationally expensive task
running computationally expensive t